In [1]:
# ### imports
import logging
import numpy as np
import pandas as pd
import tensorflow as tf

from optparse import OptionParser
from sklearn.cross_validation import train_test_split

In [2]:
# ### constant
step = 50
inf, sup = (0, step)
plitting = 0.20
LOG_FORMAT = '%(asctime)s [ %(levelname)s ] : %(message)s'

In [3]:
# ### Datasets
DATA = ( pd.DataFrame.from_csv("data/train.csv")
        .apply(lambda d: d / 255.0)
        .astype(np.float32)
       )

DATA_TRAIN, DATA_VALID = train_test_split(DATA, test_size=plitting, random_state=42)

DATA_TEST = ( pd.DataFrame.from_csv("data/test.csv", index_col=None)
             .apply(lambda d: d / 255.0)
             .astype(np.float32)
            )

In [4]:

print("Splitting the dataset at %s %%"%(plitting * 100))

print("Size of the dataset DATA_TRAIN : %s"%len(DATA_TRAIN))
print("Size of the dataset DATA_VALID : %s"%len(DATA_VALID))

print("Size of the dataset DATA_TESTS : %s"%len(DATA_TEST))

print("List of Nan values in the training set: %s"%filter(lambda y: y != 0, DATA.isnull().sum()))
print("List of Nan values in the testing  set: %s"%filter(lambda y: y != 0, DATA_TEST.isnull().sum()))


Splitting the dataset at 20.0 %
Size of the dataset DATA_TRAIN : 33600
Size of the dataset DATA_VALID : 8400
Size of the dataset DATA_TESTS : 28000
List of Nan values in the training set: []
List of Nan values in the testing  set: []


In [5]:
tf.logging.set_verbosity(tf.logging.ERROR) #enable_centered_bias=True,

feature_columns = [tf.contrib.layers.real_valued_column("", dimension=784)]


classifier = tf.contrib.learn.DNNClassifier(feature_columns=feature_columns,
                                            hidden_units=[64, 128, 128],
                                            n_classes=10,
                                            model_dir="tmp/mnist_model")


In [6]:
for i in range(1,11):
    
    classifier.fit(x=DATA_TRAIN.as_matrix() , y=DATA_TRAIN.index.tolist(), steps=2000, batch_size=50)
    
    acc_train = classifier.evaluate(x=DATA_TRAIN.as_matrix(), y=DATA_TRAIN.index.tolist())["accuracy"]
    acc_valid = classifier.evaluate(x=DATA_VALID.as_matrix(), y=DATA_VALID.index.tolist())["accuracy"]
    
    print "%s) \t Trainning  accuracy: %f"%(i*2000, acc_train)
    print "%s) \t Validation accuracy: %f"%(i*2000, acc_valid)

2000) 	 Trainning  accuracy: 0.977411
2000) 	 Validation accuracy: 0.964286
4000) 	 Trainning  accuracy: 0.987887
4000) 	 Validation accuracy: 0.969762
6000) 	 Trainning  accuracy: 0.993988
6000) 	 Validation accuracy: 0.970595
8000) 	 Trainning  accuracy: 0.996756
8000) 	 Validation accuracy: 0.969405
10000) 	 Trainning  accuracy: 0.999494
10000) 	 Validation accuracy: 0.971310
12000) 	 Trainning  accuracy: 0.999881
12000) 	 Validation accuracy: 0.972619
14000) 	 Trainning  accuracy: 0.999940
14000) 	 Validation accuracy: 0.972619
16000) 	 Trainning  accuracy: 1.000000
16000) 	 Validation accuracy: 0.972619
18000) 	 Trainning  accuracy: 1.000000
18000) 	 Validation accuracy: 0.972857
20000) 	 Trainning  accuracy: 1.000000
20000) 	 Validation accuracy: 0.973095


In [7]:
#labels = classifier.predict(x=DATA_TEST.as_matrix())

In [8]:
#df_result = pd.DataFrame({'ImageId': range(1, len(labels)+1), 'Label': labels})
#df_result.to_csv('data/prediction_six.csv', index=False)